In [ ]:
!pip install redshift_connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 811.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=ab36353508c206be03be6e4c47f4a991b3be12dea56ee25b2fb89ac803cf70e3
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [54]:
import pickle
import json
import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import unidecode
import re

from collections import Counter
from math import ceil
from langdetect import detect
from random import sample

## Exploring the ROR Data to Create Artificial Training Data

In [55]:
# Data was downloaded from the ROR website for the date seen in the file string below
# -------> https://ror.readme.io/docs/data-dump
#ror = pd.read_json("ROR_data.json")
temp_ror = pd.read_json("v1.50-2024-07-29-ror-data.json")

In [58]:
ror = temp_ror[temp_ror["country"].apply(lambda x: x.get("country_code") == "AR")]

In [60]:
ror['alias_len'] = ror['aliases'].apply(len)
ror['acronyms_len'] = ror['acronyms'].apply(len)
ror['labels_len'] = ror['labels'].apply(len)
ror['address_len'] = ror['addresses'].apply(len)
ror['address'] = ror['addresses'].apply(lambda x: x[0])
ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
ror['types'] = ror['types'].apply(lambda x: x[0])

<ipython-input-60-16646ae672a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ror['alias_len'] = ror['aliases'].apply(len)
<ipython-input-60-16646ae672a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ror['acronyms_len'] = ror['acronyms'].apply(len)
<ipython-input-60-16646ae672a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [61]:
ror[ror['ror_id']=='01r53hz59']

,id,name,types,links,aliases,acronyms,status,wikipedia_url,labels,email_address,ip_addresses,established,country,relationships,addresses,external_ids,alias_len,acronyms_len,labels_len,address_len,address,ror_id
89677,https://ror.org/01r53hz59,National University of Quilmes,Education,[http://www.unq.edu.ar/english/sections/158-unq/],[],[UNQ],active,http://en.wikipedia.org/wiki/National_University_of_Quilmes,"[{'label': 'Universidad Nacional de Quilmes', 'iso639': 'es'}]",,[],1989.0,"{'country_name': 'Argentina', 'country_code': 'AR'}",[],"[{'lat': -34.70728, 'lng': -58.2718, 'state': None, 'state_code': None, 'city': 'Bernal', 'geonames_city': {'id': 3436033, 'city': 'Bernal', 'geonames_admin1': {'name': 'Buenos Aires', 'id': 3435907, 'ascii_name': 'Buenos Aires', 'code': 'AR.01'}, 'geonames_admin2': {'name': 'Partido de Quilmes', 'id': 3429651, 'ascii_name': 'Partido de Quilmes', 'code': 'AR.01.06658'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}, 'nuts_level1': {'name': None, 'code': None}, 'nuts_level2': {'name': None, 'code': None}, 'nuts_level3': {'name': None, 'code': None}}, 'postcode': None, 'primary': False, 'line': None, 'country_geonames_id': 3865483}]","{'ISNI': {'preferred': None, 'all': ['0000 0001 1087 5626']}, 'OrgRef': {'preferred': None, 'all': ['18036946']}, 'Wikidata': {'preferred': None, 'all': ['Q6317522']}, 'GRID': {'preferred': 'grid.11560.33', 'all': 'grid.11560.33'}, 'FundRef': {'preferred': '501100009559', 'all': ['501100009559']}}",0,1,1,1,"{'lat': -34.70728, 'lng': -58.2718, 'state': None, 'state_code': None, 'city': 'Bernal', 'geonames_city': {'id': 3436033, 'city': 'Bernal', 'geonames_admin1': {'name': 'Buenos Aires', 'id': 3435907, 'ascii_name': 'Buenos Aires', 'code': 'AR.01'}, 'geonames_admin2': {'name': 'Partido de Quilmes', 'id': 3429651, 'ascii_name': 'Partido de Quilmes', 'code': 'AR.01.06658'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}, 'nuts_level1': {'name': None, 'code': None}, 'nuts_level2': {'name': None, 'code': None}, 'nuts_level3': {'name': None, 'code': None}}, 'postcode': None, 'primary': False, 'line': None, 'country_geonames_id': 3865483}",01r53hz59


In [70]:
# this file is not provided but the needed data is all institutions in OpenAlex
# with the following columns: 'ror_id','affiliation_id'
insts = pd.read_parquet("OA_static_institutions_single_file.parquet",
                       columns=['affiliation_id','ror_id'])

#insts = pd.read_json("OA_data.json")

In [71]:
insts.head()

,affiliation_id,ror_id
0,https://openalex.org/I151201029,https://ror.org/03cqe8w59
1,https://openalex.org/I24354313,https://ror.org/0081fs513
2,https://openalex.org/I874386039,https://ror.org/01tjs6929
3,https://openalex.org/I166401450,https://ror.org/056tb7j80
4,https://openalex.org/I53241121,https://ror.org/05rxmkq09


In [73]:
insts['ror_id'] = insts['ror_id'].apply(lambda x: x.split("/")[-1])
insts['affiliation_id'] = insts['affiliation_id'].apply(lambda  x: x.split("I")[-1])
insts= insts[['affiliation_id', 'ror_id']]
insts.head()

,affiliation_id,ror_id
0,151201029,03cqe8w59
1,24354313,0081fs513
2,874386039,01tjs6929
3,166401450,056tb7j80
4,53241121,05rxmkq09


In [74]:
ror_to_join = ror[['ror_id','name','aliases','acronyms','labels','country','types',
                   'address','alias_len','acronyms_len','labels_len','address_len']].copy()

In [75]:
def get_geoname_admin(address_dict):
    try:
        geoname_admin = address_dict['geonames_city']['geonames_admin1']['name']
    except:
        geoname_admin = "None"

    return geoname_admin

In [76]:
ror_to_join['country_code'] = ror_to_join['country'].apply(lambda x: x['country_code'])
ror_to_join['country_name'] = ror_to_join['country'].apply(lambda x: x['country_name'])
ror_to_join['city'] = ror_to_join['address'].apply(lambda x: x['city'])
ror_to_join['state'] = ror_to_join['address'].apply(lambda x: x['state'])
ror_to_join['region'] = ror_to_join['address'].apply(get_geoname_admin)
ror_to_join['institution'] = ror_to_join['name']

##### Looking at ROR Labels

In [77]:
codes_to_ignore = ['ja','fa','hi','ko','bn','zh','ml','ru','el','kn','gu','mk','ne','te','hy',
                   'km','ti','kk','th','my','uk','pa','bg','ur','vi','ar','sr','he','ta','ka',
                   'am','mr','lo','mn','be','or','ba','si','ky','uz']

In [78]:
labels = ror_to_join['labels'].explode().dropna().reset_index()
labels['label'] = labels['labels'].apply(lambda x: x['label'])
labels['iso639'] = labels['labels'].apply(lambda x: x['iso639'])

In [79]:
labels[~labels['iso639'].isin(codes_to_ignore)].sample(20)

,index,labels,label,iso639
83,92354,"{'label': 'Universidad Nacional de Avellaneda', 'iso639': 'es'}",Universidad Nacional de Avellaneda,es
85,92841,"{'label': 'Fundación INFANT', 'iso639': 'es'}",Fundación INFANT,es
52,65092,"{'label': 'Cardiovascular Institute of Rosario', 'iso639': 'en'}",Cardiovascular Institute of Rosario,en
33,26118,"{'label': 'Universidad Nacional de La Rioja', 'iso639': 'es'}",Universidad Nacional de La Rioja,es
102,98264,"{'label': 'Instituto de Investigación en Biomedicina de Buenos Aires - Instituto Partner de la Sociedad Max Planck', 'iso639': 'es'}",Instituto de Investigación en Biomedicina de Buenos Aires - Instituto Partner de la Sociedad Max Planck,es
121,102076,"{'label': 'Latin American Council of Social Sciences', 'iso639': 'en'}",Latin American Council of Social Sciences,en
19,16984,"{'label': 'Catholic University of Cuyo', 'iso639': 'en'}",Catholic University of Cuyo,en
86,93158,"{'label': 'Instituto Balseiro', 'iso639': 'es'}",Instituto Balseiro,es
99,97763,"{'label': 'Katholische Universität Córdoba', 'iso639': 'de'}",Katholische Universität Córdoba,de
51,59582,"{'label': 'Instituto de Física Rosario', 'iso639': 'es'}",Instituto de Física Rosario,es


#### Getting string beginnings

Looking to introduce more variety into the artificial strings so that they include some header information such as "College of .." or "Department of...".

In [80]:
with open("Company", 'r') as f:
    company_begs = f.readlines()

company_begs = list(set([x.rstrip('\n') for x in company_begs]))

In [81]:
with open("Education_dept", 'r') as f:
    education_dept_begs = f.readlines()

education_dept_begs = list(set([x.rstrip('\n') for x in education_dept_begs]))

In [82]:
with open("Education_college", 'r') as f:
    education_col_begs = f.readlines()

education_col_begs = list(set([x.rstrip('\n') for x in education_col_begs]))

In [83]:
with open("Education_school", 'r') as f:
    education_school_begs = f.readlines()

education_school_begs = list(set([x.rstrip('\n') for x in education_school_begs]))

In [84]:
with open("Healthcare", 'r') as f:
    healthcare_begs = f.readlines()

healthcare_begs = list(set([x.rstrip('\n') for x in healthcare_begs]))

In [85]:
all_education = []
for beg in education_col_begs:
    all_education.append(f"College of {beg}")

for beg in education_dept_begs:
    all_education.append(f"Department of {beg}")
    all_education.append(f"Dep of {beg}")
    all_education.append(f"Dept of {beg}")
    all_education.append(f"Dep. of {beg}")
    all_education.append(f"Dept. of {beg}")

for beg in education_school_begs:
    all_education.append(f"School of {beg}")
    all_education.append(f"Sch. of {beg}")
    all_education.append(f"Sch of {beg}")
len(all_education)

272

In [86]:
all_company = company_begs

In [87]:
all_healthcare = []
for beg in healthcare_begs:
    all_healthcare.append(f"Department of {beg}")
    all_healthcare.append(f"Dep of {beg}")
    all_healthcare.append(f"Dept of {beg}")
    all_healthcare.append(f"Dep. of {beg}")
    all_healthcare.append(f"Dept. of {beg}")

##### Creating the artificial affiliation strings

We would like to take advantage of the ROR data and use it to supplement/augment the current affiliation string data in OpenAlex. This could potentially allow for the Institution Tagger model to predict on institutions that have not yet had affiliation strings added to OpenAlex.

In [88]:
type_preinst_dict = {'Company': all_company,
                     'Education': all_education,
                     'Healthcare': all_healthcare}

In [89]:
def create_affiliation_string_from_scratch(institution, city, state, country, region, aliases, labels,
                                           acronyms, inst_type):
    aff_strings = []
    if aliases:
        aliases = [institution] + aliases
    else:
        aliases = [institution]
    if labels:
        for label in labels:
            if label['iso639'] in codes_to_ignore:
                pass
            else:
                aliases.append(label['label'])
    if acronyms:
        for acronym in acronyms:
            aliases.append(acronym)
    for alias in aliases:
        if "r&d" not in alias.lower():
            alias = alias.replace(" & ", " and ")
        match_string = unidecode.unidecode(alias)
        match_string = match_string.lower().replace(" ", "")
        match_string = "".join(re.findall("[a-zA-Z0-9]+", match_string))
        alias = unidecode.unidecode(alias)
        if ((state is None) & (region != city) & (city is not None) &
            (country is not None) & (region is not None)):
            region = unidecode.unidecode(region)
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {region}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {region}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias}, {region}", match_string])
            aff_strings.append([f"{alias} {city} {region} {country}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {city} {region}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias} {region}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(5):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(2):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])

        elif (state is not None) & (city is not None) & (country is not None):
            state = unidecode.unidecode(state)
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {state}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {state}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias}, {state}", match_string])
            aff_strings.append([f"{alias} {city} {state} {country}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {city} {state}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias} {state}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
        elif (city is not None) & (country is not None):
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
        elif (country is not None):
            country = unidecode.unidecode(country)
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(4):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])

        else:
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(5):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                else:
                    for i in range(2):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
    return aff_strings

In [90]:
def get_institutions_list(institution, aliases, labels, acronyms):
    aff_strings = []
    if aliases:
        aliases = [institution] + aliases
    else:
        aliases = [institution]
    if labels:
        for label in labels:
            if label['iso639'] in codes_to_ignore:
                pass
            else:
                aliases.append(label['label'])
    return aliases

In [91]:
ror_to_join['affs'] = ror_to_join \
.apply(lambda x: get_institutions_list(x.institution, x.aliases,
                                       x.labels, x.acronyms), axis=1)
ror_to_join['aff_string'] = ror_to_join \
.apply(lambda x: create_affiliation_string_from_scratch(x.institution, x.city, x.state,
                                                        x.country_name, x.region,
                                                        x.aliases, x.labels, x.acronyms, x.types), axis=1)
ror_to_join['aff_string_len'] = ror_to_join['aff_string'].apply(len)
ror_to_join_final = ror_to_join.explode("aff_string").copy()

##### Looking to quickly get combinations of city/region/country

In [92]:
art_empty_affs = ror_to_join[['city','region','country_name']].dropna().copy()

In [93]:
art_empty_affs.shape

(347, 3)

In [94]:
art_empty_affs.sample(10)

,city,region,country_name
77724,Buenos Aires,Buenos Aires F.D.,Argentina
93493,Puerto Iguazú,Misiones,Argentina
98393,Buenos Aires,Buenos Aires F.D.,Argentina
98290,Córdoba,Cordoba,Argentina
76467,Bahía Blanca,Buenos Aires,Argentina
95912,Buenos Aires,Buenos Aires F.D.,Argentina
96587,Córdoba,Cordoba,Argentina
26087,Buenos Aires,Buenos Aires F.D.,Argentina
51827,Buenos Aires,Buenos Aires F.D.,Argentina
51721,San Miguel de Tucumán,Tucuman,Argentina


In [95]:
art_empty_affs['original_affiliation_1'] = \
    art_empty_affs.apply(lambda x: f"{x.city}, {x.country_name}", axis=1)

In [96]:
art_empty_affs['original_affiliation_2'] = \
    art_empty_affs.apply(lambda x: f"{x.city}, {x.region}, {x.country_name}", axis=1)

In [97]:
city_country = art_empty_affs.sample(50).drop_duplicates()\
    ['original_affiliation_1'].to_list()

In [98]:
city_region_country = art_empty_affs.sample(50).drop_duplicates()\
    ['original_affiliation_2'].to_list()

Some of these string will be used to train the model that only seeing a city/region/country should be a "no prediction"

In [99]:
pd.DataFrame(zip(city_country+city_region_country),
             columns=['original_affiliation']) \
    .drop_duplicates(subset=['original_affiliation'])\
    .to_parquet("artificial_empty_affs.parquet")

In [100]:
ror_to_join_final = ror_to_join[['ror_id','aff_string']].explode("aff_string").copy()
ror_to_join_final.shape

(51430, 2)

In [101]:
ror_to_join_final['original_affiliation'] = \
    ror_to_join_final['aff_string'].apply(lambda x: x[0])

The rest of the artificial strings are exported so that they can be combined with the historical affiliation data to create the final training data set.

In [102]:
ror_to_join_final.merge(insts, how='inner',
                        on='ror_id')[['original_affiliation','affiliation_id']] \
.to_parquet("ror_strings.parquet")